# Importing Libraries

In [50]:
import numpy as np 
import pandas as pd
import matplotlib as plt
from datetime import timedelta

# Assinging Data Types

In [2]:
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}

# Reading Datasets

In [3]:
train = pd.read_csv('train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    )

# Renaming Column name 

In [4]:
holiday =  pd.read_csv('holidays_events.csv')
holiday = holiday.loc[holiday['transferred'] == False]
holiday =holiday.rename(columns = {'locale_name':'city'})

# Removing Negative entries

In [5]:
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

C:\Users\ARNABD~1\AppData\Local\Temp/ipykernel_21300/140917293.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion


# Assingning Dates , Store Numbers and Item Numbers

In [6]:
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

In [7]:
del u_dates, u_stores, u_items

# Removing Null Values

In [8]:
train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date


In [9]:
test = pd.read_csv('test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek

# Moving Average

## Creation of Moving Average model and merging with Training Data Set to create a new training dataset

In [10]:
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

In [11]:
print(ma_is)

                     mais226   mais112    mais56    mais28    mais14  \
item_nbr store_nbr                                                     
96995    1          0.106638  0.106638  0.106638  0.106638  0.106638   
         2          0.222336  0.222336  0.222336  0.222336  0.222336   
         3          0.000000  0.000000  0.000000  0.000000  0.000000   
         4          0.222336  0.222336  0.222336  0.222336  0.222336   
         5          0.053319  0.053319  0.053319  0.053319  0.053319   
...                      ...       ...       ...       ...       ...   
1118683  48         0.191147  0.191147  0.191147  0.191147  0.191147   
         49         0.106638  0.106638  0.106638  0.106638  0.106638   
         50         0.191147  0.191147  0.191147  0.191147  0.191147   
         51         0.351104  0.351104  0.351104  0.351104  0.351104   
         54         0.000000  0.000000  0.000000  0.000000  0.000000   

                       mais7     mais3     mais1  
item_nbr sto

In [12]:
ma_is.head()

mais226   mais112    mais56    mais28    mais14  \
item_nbr store_nbr                                                     
96995    1          0.106638  0.106638  0.106638  0.106638  0.106638   
         2          0.222336  0.222336  0.222336  0.222336  0.222336   
         3          0.000000  0.000000  0.000000  0.000000  0.000000   
         4          0.222336  0.222336  0.222336  0.222336  0.222336   
         5          0.053319  0.053319  0.053319  0.053319  0.053319   

                       mais7     mais3     mais1  
item_nbr store_nbr                                
96995    1          0.115525  0.115525  0.115525  
         2          0.240864  0.240864  0.240864  
         3          0.000000  0.000000  0.000000  
         4          0.240864  0.240864  0.240864  
         5          0.057762  0.057762  0.057762

In [13]:
ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)

In [14]:
ma_is.head()

,item_nbr,store_nbr,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,96995,1,0.106638,0.106638,0.106638,0.106638,0.106638,0.115525,0.115525,0.115525,0.106638
1,96995,2,0.222336,0.222336,0.222336,0.222336,0.222336,0.240864,0.240864,0.240864,0.222336
2,96995,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,96995,4,0.222336,0.222336,0.222336,0.222336,0.222336,0.240864,0.240864,0.240864,0.222336
4,96995,5,0.053319,0.053319,0.053319,0.053319,0.053319,0.057762,0.057762,0.057762,0.053319


In [15]:
train.head()

,date,store_nbr,item_nbr,unit_sales,dow
0,2013-01-01,25,103665,2.079442,1.0
1,2013-01-01,25,105574,0.693147,1.0
2,2013-01-01,25,105575,1.098612,1.0
3,2013-01-01,25,108079,0.693147,1.0
4,2013-01-01,25,108701,0.693147,1.0


In [16]:
train1=pd.merge(train,ma_is)

In [17]:
train1.head()

,date,store_nbr,item_nbr,unit_sales,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,2013-01-01,25,103665,2.079442,1.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
1,2013-02-01,25,103665,1.791759,4.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
2,2013-03-01,25,103665,0.000000,NaN,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
3,2013-04-01,25,103665,1.791759,0.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
4,2013-05-01,25,103665,1.791759,2.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855


In [18]:
df=train1

In [20]:
df.head()

,date,store_nbr,item_nbr,unit_sales,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,2013-01-01,25,103665,2.079442,1.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
1,2013-02-01,25,103665,1.791759,4.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
2,2013-03-01,25,103665,0.000000,NaN,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
3,2013-04-01,25,103665,1.791759,0.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
4,2013-05-01,25,103665,1.791759,2.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855


In [25]:
df.drop("mais226", axis=1 )

,date,store_nbr,item_nbr,unit_sales,dow,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,2013-01-01,25,103665,2.079442,1.0,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
1,2013-02-01,25,103665,1.791759,4.0,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
2,2013-03-01,25,103665,0.000000,NaN,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
3,2013-04-01,25,103665,1.791759,0.0,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
4,2013-05-01,25,103665,1.791759,2.0,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960981,2013-09-01,54,1114566,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
960982,2013-10-01,54,1114566,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
960983,2013-11-01,54,1114566,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
960984,2013-12-01,54,1114566,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
df.drop("dow",axis=1)

,date,store_nbr,item_nbr,unit_sales,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,2013-01-01,25,103665,2.079442,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
1,2013-02-01,25,103665,1.791759,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
2,2013-03-01,25,103665,0.000000,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
3,2013-04-01,25,103665,1.791759,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
4,2013-05-01,25,103665,1.791759,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960981,2013-09-01,54,1114566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
960982,2013-10-01,54,1114566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
960983,2013-11-01,54,1114566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
960984,2013-12-01,54,1114566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [34]:
dtypes = {'mais':'float'}

In [35]:
train_final=df

# Final Dataset

In [37]:
train_final.head()

,date,store_nbr,item_nbr,unit_sales,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,2013-01-01,25,103665,2.079442,1.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
1,2013-02-01,25,103665,1.791759,4.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
2,2013-03-01,25,103665,0.000000,NaN,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
3,2013-04-01,25,103665,1.791759,0.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
4,2013-05-01,25,103665,1.791759,2.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855


# Applying Linear Regression by taking store number and item number as X parameter and moving aversge ( 7 day period as Y parameter )

In [47]:
df = train_final

X = df[['store_nbr','item-nbr']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
Y = df['unit_sales']

# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 -1.6631140908884845e-13
Coefficients: 
 [-8.22299554e-17  1.00000000e+00]


# Exporting Final Dataset in csv format for GUI

In [48]:
train_final.head()

,date,store_nbr,item_nbr,unit_sales,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais
0,2013-01-01,25,103665,2.079442,1.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
1,2013-02-01,25,103665,1.791759,4.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
2,2013-03-01,25,103665,0.000000,NaN,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
3,2013-04-01,25,103665,1.791759,0.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855
4,2013-05-01,25,103665,1.791759,2.0,1.132855,1.132855,1.132855,1.132855,1.132855,1.053973,1.053973,1.053973,1.132855


In [51]:
train_final.to_csv('finaldatasetforgui.')